In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install gspread
!pip install oauth2client

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import gspread
from google.colab import auth
import pandas as pd
import math

# Open the Google Sheet by its URL
sheet_url = 'https://docs.google.com/spreadsheets/d/1Yi_xPdR-ioa9HpkYFU7lW4bcarz3OZahH-91xMiXbJI/edit?usp=sharing'
worksheet = gc.open_by_url(sheet_url).sheet1

# Get all records from the sheet
records = worksheet.get_all_records()

# Convert records to DataFrame
ndf = pd.DataFrame(records)


In [ ]:
#/content/drive/MyDrive/BFI-2 (Responses) - Formresponses1.csv

In [ ]:
ndf

,Timestamp,Name:,"I am someone who is outgoing, sociable.","I am someone who is compassionate, has a soft heart.",I am someone who tends to be disorganized.,"I am someone who is relaxed, handles stress well.",I am someone who has few artistic interests.,I am someone who has an assertive personality.,"I am someone who is respectful, treats others with respect.",I am someone who tends to be lazy.,...,Trust,Organization,Productiveness,Responsibility,Anxiety,Depression,Emotional Volatility,Intellectual Curiosity,Aesthetic Sensitivity,Creative Imagination
0,23/05/2024 12:37:04,Masum,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Strongly Agree,Neutral,...,High,High,High,High,Medium,Low,Low,High,High,High
1,hfkejenkj,iii,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,,,,,,,,,,


In [ ]:
df = ndf.iloc[:1,2:]
df

,"I am someone who is outgoing, sociable.","I am someone who is compassionate, has a soft heart.",I am someone who tends to be disorganized.,"I am someone who is relaxed, handles stress well.",I am someone who has few artistic interests.,I am someone who has an assertive personality.,"I am someone who is respectful, treats others with respect.",I am someone who tends to be lazy.,I am someone who stays optimistic after experiencing a setback.,I am someone who is curious about many different things.,...,Trust,Organization,Productiveness,Responsibility,Anxiety,Depression,Emotional Volatility,Intellectual Curiosity,Aesthetic Sensitivity,Creative Imagination
0,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Strongly Agree,Neutral,Agree,Neutral,...,High,High,High,High,Medium,Low,Low,High,High,High


In [ ]:
# Define mapping for converting responses to numeric values
response_mapping = {
    'Strongly Disagree': 1,
    'Disagree': 2,
    'Neutral': 3,
    'Agree': 4,
    'Strongly Agree': 5
}

# Define function to convert response to numeric value
def convert_response_to_numeric(response):
    return response_mapping.get(response, 0)

# Define function to handle reverse scoring
def handle_reverse_scoring(score, reverse):
    return 6 - score if reverse else score

# Calculate sum for domain scales
def calculate_domain_scale_sum(df, scales):
    scale_sum = {}
    for scale, questions in scales.items():
        total = 0
        for question in questions:
            question_index = int(question[:-1]) if question.endswith('R') else int(question)
            reverse = question.endswith('R')
            total += handle_reverse_scoring(convert_response_to_numeric(df.iloc[:, question_index - 1]), reverse)
        scale_sum[scale] = total
    return scale_sum

# Calculate sum for facet scales
def calculate_facet_scale_sum(df, scales):
    scale_sum = {}
    for scale, questions in scales.items():
        total = 0
        for question in questions:
            question_index = int(question[:-1]) if question.endswith('R') else int(question)
            reverse = question.endswith('R')
            total += handle_reverse_scoring(convert_response_to_numeric(df.iloc[:, question_index - 1]), reverse)
        scale_sum[scale] = total
    return scale_sum

# Domain scales
domain_scales = {
    'Extraversion': ['1', '6', '11R', '16R', '21', '26R', '31R', '36R', '41', '46', '51R', '56'],
    'Agreeableness': ['2', '7', '12R', '17R', '22R', '27', '32', '37R', '42R', '47R', '52', '57'],
    'Conscientiousness': ['3R', '8R', '13', '18', '23R', '28R', '33', '38', '43', '48R', '53', '58R'],
    'Negative Emotionality': ['4R', '9R', '14', '19', '24R', '29R', '34', '39', '44R', '49R', '54', '59'],
    'Open-Mindedness': ['5R', '10', '15', '20', '25R', '30R', '35', '40', '45R', '50R', '55R', '60']
}

# Facet scales
facet_scales = {
    'Sociability': ['1', '16R', '31R', '46'],
    'Assertiveness': ['6', '21', '36R', '51R'],
    'Energy Level': ['11R', '26R', '41', '56'],
    'Compassion': ['2', '17R', '32', '47R'],
    'Respectfulness': ['7', '22R', '37R', '52'],
    'Trust': ['12R', '27', '42R', '57'],
    'Organization': ['3R', '18', '33', '48R'],
    'Productiveness': ['8R', '23R', '38', '53'],
    'Responsibility': ['13', '28R', '43', '58R'],
    'Anxiety': ['4R', '19', '34', '49R'],
    'Depression': ['9R', '24R', '39', '54'],
    'Emotional Volatility': ['14', '29R', '44R', '59'],
    'Intellectual Curiosity': ['10', '25R', '40', '55R'],
    'Aesthetic Sensitivity': ['5R', '20', '35', '50R'],
    'Creative Imagination': ['15', '30R', '45R', '60']
}
def calculate_domain_scale_sum(df, scales):
    scale_sum = {}
    for scale, questions in scales.items():
        questions.sort(key=lambda x: int(x[:-1]) if x.endswith('R') else int(x))
        total = 0
        for question in questions:
            question_index = int(question[:-1]) if question.endswith('R') else int(question)
            response = df.iloc[:, question_index - 1]
            # Debug print statements
            #print(f"Question: {question}, Question Index: {question_index}, Response: {response}")
            score = response.map(response_mapping).fillna(0)
            # Debug print statements
            #print(f"Original Score: {score}")

            if question.endswith('R'):
                score = 6 - score
                # Debug print statements
             #   print(f"Reversed Score: {score}")

            total += score.sum()

        scale_sum[scale] = total
    return scale_sum

# Define function to calculate sum for facet scales
def calculate_facet_scale_sum(df, scales):
    scale_sum = {}
    for scale, questions in scales.items():
        total = 0
        for question in questions:
            question_index = int(question[:-1]) if question.endswith('R') else int(question)
            response = df.iloc[:, question_index - 1]
            score = response.map(response_mapping).fillna(0)
            if question.endswith('R'):
                score = 6 - score
            total += score.sum()
        scale_sum[scale] = total
    return scale_sum

# Calculate sum of scores for domain scales and facet scales
domain_scale_sum = calculate_domain_scale_sum(df, domain_scales)
facet_scale_sum = calculate_facet_scale_sum(df, facet_scales)

# Divide sum of domain scales by 12 and sum of facet scales by 4, then round

def custom_round(x):
    """Custom rounding function to round halfway values up"""
    integer_part = int(x)
    decimal_part = x - integer_part
    if decimal_part >= 0.5:
        return integer_part + 1
    else:
        return round(x)

domain_scale_avg = {scale: custom_round(total / 12) for scale, total in domain_scale_sum.items()}

facet_scale_avg = {scale: custom_round(total / 4) for scale, total in facet_scale_sum.items()}


# Define function to map numeric scores to categories
def map_to_category(score):
    if score == 1:
        return "Very Low"
    elif score == 2:
        return "Low"
    elif score == 3:
        return "Medium"
    elif score == 4:
        return "High"
    elif score == 5:
        return "Very High"
    else:
        return "Unknown"

# Map domain scale averages to categories
domain_scale_categories = {scale: map_to_category(avg) for scale, avg in domain_scale_avg.items()}

# Map facet scale averages to categories
facet_scale_categories = {scale: map_to_category(avg) for scale, avg in facet_scale_avg.items()}

# Print results
print("Domain Scale Categories:")
for scale, category in domain_scale_categories.items():
    print(f"{scale}: {category}")

print("\nFacet Scale Categories:")
for scale, category in facet_scale_categories.items():
    print(f"{scale}: {category}")


Domain Scale Categories:
Extraversion: Medium
Agreeableness: High
Conscientiousness: High
Negative Emotionality: Low
Open-Mindedness: High

Facet Scale Categories:
Sociability: Low
Assertiveness: Medium
Energy Level: Medium
Compassion: High
Respectfulness: High
Trust: High
Organization: High
Productiveness: High
Responsibility: High
Anxiety: Medium
Depression: Low
Emotional Volatility: Low
Intellectual Curiosity: High
Aesthetic Sensitivity: High
Creative Imagination: High


In [ ]:
# Map facet scale averages to categories
facet_scale_categories = {scale: map_to_category(avg) for scale, avg in facet_scale_avg.items()}

# Merge domain and facet scale categories into a DataFrame
results_df = pd.DataFrame({
    'Domain Scales': domain_scale_categories,
    'Facet Scales': facet_scale_categories
})

# Print results
print(results_df)

                       Domain Scales Facet Scales
Extraversion                  Medium          NaN
Agreeableness                   High          NaN
Conscientiousness               High          NaN
Negative Emotionality            Low          NaN
Open-Mindedness                 High          NaN
Sociability                      NaN          Low
Assertiveness                    NaN       Medium
Energy Level                     NaN       Medium
Compassion                       NaN         High
Respectfulness                   NaN         High
Trust                            NaN         High
Organization                     NaN         High
Productiveness                   NaN         High
Responsibility                   NaN         High
Anxiety                          NaN       Medium
Depression                       NaN          Low
Emotional Volatility             NaN          Low
Intellectual Curiosity           NaN         High
Aesthetic Sensitivity            NaN         High


In [ ]:

# Convert records to DataFrame
ndf = pd.DataFrame(records)
df = ndf.iloc[:, 2:]  # Assuming the first two columns are Timestamp and Name

# Define mapping for converting responses to numeric values
response_mapping = {
    'Strongly Disagree': 1,
    'Disagree': 2,
    'Neutral': 3,
    'Agree': 4,
    'Strongly Agree': 5
}

# Define function to convert response to numeric value
def convert_response_to_numeric(response):
    return response_mapping.get(response, 0)

# Define function to handle reverse scoring
def handle_reverse_scoring(score, reverse):
    return 6 - score if reverse else score

# Define scales
domain_scales = {
    'Extraversion': ['1', '6', '11R', '16R', '21', '26R', '31R', '36R', '41', '46', '51R', '56'],
    'Agreeableness': ['2', '7', '12R', '17R', '22R', '27', '32', '37R', '42R', '47R', '52', '57'],
    'Conscientiousness': ['3R', '8R', '13', '18', '23R', '28R', '33', '38', '43', '48R', '53', '58R'],
    'Negative Emotionality': ['4R', '9R', '14', '19', '24R', '29R', '34', '39', '44R', '49R', '54', '59'],
    'Open-Mindedness': ['5R', '10', '15', '20', '25R', '30R', '35', '40', '45R', '50R', '55R', '60']
}

facet_scales = {
    'Sociability': ['1', '16R', '31R', '46'],
    'Assertiveness': ['6', '21', '36R', '51R'],
    'Energy Level': ['11R', '26R', '41', '56'],
    'Compassion': ['2', '17R', '32', '47R'],
    'Respectfulness': ['7', '22R', '37R', '52'],
    'Trust': ['12R', '27', '42R', '57'],
    'Organization': ['3R', '18', '33', '48R'],
    'Productiveness': ['8R', '23R', '38', '53'],
    'Responsibility': ['13', '28R', '43', '58R'],
    'Anxiety': ['4R', '19', '34', '49R'],
    'Depression': ['9R', '24R', '39', '54'],
    'Emotional Volatility': ['14', '29R', '44R', '59'],
    'Intellectual Curiosity': ['10', '25R', '40', '55R'],
    'Aesthetic Sensitivity': ['5R', '20', '35', '50R'],
    'Creative Imagination': ['15', '30R', '45R', '60']
}

# Define function to calculate sum for domain scales
def calculate_scale_sum(row, scales):
    scale_sum = {}
    for scale, questions in scales.items():
        total = 0
        for question in questions:
            question_index = int(question[:-1]) - 1 if question.endswith('R') else int(question) - 1
            reverse = question.endswith('R')
            response = row.iloc[question_index]
            score = convert_response_to_numeric(response)
            score = handle_reverse_scoring(score, reverse)
            total += score
        scale_sum[scale] = total
    return scale_sum

# Define function to map numeric scores to categories
def map_to_category(score):
    if score == 1:
        return "Very Low"
    elif score == 2:
        return "Low"
    elif score == 3:
        return "Medium"
    elif score == 4:
        return "High"
    else:
        return "Very High"

# Calculate averages for domain scales
domain_scale_avgs = []
for _, row in df.iterrows():
    domain_sums = calculate_scale_sum(row, domain_scales)
    domain_avgs = {scale: total / 12 for scale, total in domain_sums.items()}
    domain_categories = {scale: map_to_category(avg) for scale, avg in domain_avgs.items()}
    domain_scale_avgs.append(domain_categories)

# Calculate averages for facet scales
facet_scale_avgs = []
for _, row in df.iterrows():
    facet_sums = calculate_scale_sum(row, facet_scales)
    facet_avgs = {scale: total / 4 for scale, total in facet_sums.items()}
    facet_categories = {scale: map_to_category(avg) for scale, avg in facet_avgs.items()}
    facet_scale_avgs.append(facet_categories)

# Merge domain and facet scale categories into a DataFrame
merged_categories = []
for domain, facet in zip(domain_scale_avgs, facet_scale_avgs):
    merged_categories.append({**domain, **facet})

# Convert the merged categories to a DataFrame
results_df = pd.DataFrame(merged_categories)

# Add other columns (e.g., Name) from the original DataFrame
results_df = pd.concat([ndf[['Name:']], results_df], axis=1)

# Print results
print(results_df)


   Name: Extraversion Agreeableness Conscientiousness Negative Emotionality  \
0  Masum    Very High     Very High         Very High             Very High   

  Open-Mindedness Sociability Assertiveness Energy Level Compassion  ...  \
0       Very High         Low        Medium       Medium  Very High  ...   

  Trust Organization Productiveness Responsibility    Anxiety Depression  \
0  High    Very High      Very High           High  Very High        Low   

  Emotional Volatility Intellectual Curiosity Aesthetic Sensitivity  \
0            Very High              Very High                  High   

  Creative Imagination  
0            Very High  

[1 rows x 21 columns]


In [ ]:

# Load the Google Sheet
sheet_url = 'https://docs.google.com/spreadsheets/d/1Yi_xPdR-ioa9HpkYFU7lW4bcarz3OZahH-91xMiXbJI/edit?usp=sharing'
worksheet = gc.open_by_url(sheet_url).sheet1

# Get all records from the sheet
records = worksheet.get_all_records()

# Convert records to DataFrame
ndf = pd.DataFrame(records)
df = ndf.iloc[:, 2:]  # Assuming the first two columns are Timestamp and Name

# Define mapping for converting responses to numeric values
response_mapping = {
    'Strongly Disagree': 1,
    'Disagree': 2,
    'Neutral': 3,
    'Agree': 4,
    'Strongly Agree': 5
}

# Define function to convert response to numeric value
def convert_response_to_numeric(response):
    return response_mapping.get(response, 0)

# Define function to handle reverse scoring
def handle_reverse_scoring(score, reverse):
    return 6 - score if reverse else score

# Define scales
domain_scales = {
    'Extraversion': ['1', '6', '11R', '16R', '21', '26R', '31R', '36R', '41', '46', '51R', '56'],
    'Agreeableness': ['2', '7', '12R', '17R', '22R', '27', '32', '37R', '42R', '47R', '52', '57'],
    'Conscientiousness': ['3R', '8R', '13', '18', '23R', '28R', '33', '38', '43', '48R', '53', '58R'],
    'Negative Emotionality': ['4R', '9R', '14', '19', '24R', '29R', '34', '39', '44R', '49R', '54', '59'],
    'Open-Mindedness': ['5R', '10', '15', '20', '25R', '30R', '35', '40', '45R', '50R', '55R', '60']
}

facet_scales = {
    'Sociability': ['1', '16R', '31R', '46'],
    'Assertiveness': ['6', '21', '36R', '51R'],
    'Energy Level': ['11R', '26R', '41', '56'],
    'Compassion': ['2', '17R', '32', '47R'],
    'Respectfulness': ['7', '22R', '37R', '52'],
    'Trust': ['12R', '27', '42R', '57'],
    'Organization': ['3R', '18', '33', '48R'],
    'Productiveness': ['8R', '23R', '38', '53'],
    'Responsibility': ['13', '28R', '43', '58R'],
    'Anxiety': ['4R', '19', '34', '49R'],
    'Depression': ['9R', '24R', '39', '54'],
    'Emotional Volatility': ['14', '29R', '44R', '59'],
    'Intellectual Curiosity': ['10', '25R', '40', '55R'],
    'Aesthetic Sensitivity': ['5R', '20', '35', '50R'],
    'Creative Imagination': ['15', '30R', '45R', '60']
}

# Define function to calculate sum for scales
def calculate_scale_sum(row, scales):
    scale_sum = {}
    for scale, questions in scales.items():
        total = 0
        for question in questions:
            question_index = int(question[:-1]) - 1 if question.endswith('R') else int(question) - 1
            reverse = question.endswith('R')
            response = row.iloc[question_index]
            score = convert_response_to_numeric(response)
            score = handle_reverse_scoring(score, reverse)
            total += score
        scale_sum[scale] = total
    return scale_sum

# Define function to map numeric scores to categories
def map_to_category(score):
    if score == 1:
        return "Very Low"
    elif score == 2:
        return "Low"
    elif score == 3:
        return "Medium"
    elif score == 4:
        return "High"
    elif score == 5:
        return "Very High"
    else:
        return "Unknown"
def custom_round(x):
    """Custom rounding function to round halfway values up"""
    integer_part = int(x)
    decimal_part = x - integer_part
    if decimal_part >= 0.5:
        return integer_part + 1
    else:
        return round(x)

# Process each row to calculate domain and facet scale categories
results = []
for index, row in df.iterrows():
    domain_sums = calculate_scale_sum(row, domain_scales)
    domain_avgs = {scale: total / 12 for scale, total in domain_sums.items()}
    domain_categories = {scale: map_to_category(custom_round(avg)) for scale, avg in domain_avgs.items()}

    facet_sums = calculate_scale_sum(row, facet_scales)
    facet_avgs = {scale: total / 4 for scale, total in facet_sums.items()}
    facet_categories = {scale: map_to_category(custom_round(avg)) for scale, avg in facet_avgs.items()}

    result = {**domain_categories, **facet_categories}
    result['Name:'] = ndf.loc[index, 'Name:']
    results.append(result)

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Print results
results_df


,Extraversion,Agreeableness,Conscientiousness,Negative Emotionality,Open-Mindedness,Sociability,Assertiveness,Energy Level,Compassion,Respectfulness,...,Organization,Productiveness,Responsibility,Anxiety,Depression,Emotional Volatility,Intellectual Curiosity,Aesthetic Sensitivity,Creative Imagination,Name:
0,Medium,High,High,Low,High,Low,Medium,Medium,High,High,...,High,High,High,Medium,Low,Low,High,High,High,Masum
1,Medium,Medium,Medium,Medium,Medium,Medium,Medium,High,Medium,Medium,...,Medium,Medium,Medium,Medium,High,Medium,Medium,Medium,Medium,jfoi
2,Medium,High,High,Low,High,Low,Medium,Medium,High,High,...,High,High,High,Medium,Low,Low,High,High,High,Masum
3,Medium,Medium,Medium,Medium,Medium,Medium,Medium,High,Medium,Medium,...,Medium,Medium,Medium,Medium,High,Medium,Medium,Medium,Medium,jfoi


In [ ]:
results_df

,Extraversion,Agreeableness,Conscientiousness,Negative Emotionality,Open-Mindedness,Sociability,Assertiveness,Energy Level,Compassion,Respectfulness,...,Organization,Productiveness,Responsibility,Anxiety,Depression,Emotional Volatility,Intellectual Curiosity,Aesthetic Sensitivity,Creative Imagination,Name:
0,Medium,High,High,Low,High,Low,Medium,Medium,High,High,...,High,High,High,Medium,Low,Low,High,High,High,Masum
1,Medium,Medium,Medium,Medium,Medium,Medium,Medium,High,Medium,Medium,...,Medium,Medium,Medium,Medium,High,Low,Low,Medium,Medium,iiii


In [ ]:
import pandas as pd

# Read the Google Sheets data
sheet_url = 'https://docs.google.com/spreadsheets/d/1Yi_xPdR-ioa9HpkYFU7lW4bcarz3OZahH-91xMiXbJI/edit?usp=sharing'
df = ndf


# Define mapping for converting responses to numeric values
response_mapping = {
    'Strongly Disagree': 1,
    'Disagree': 2,
    'Neutral': 3,
    'Agree': 4,
    'Strongly Agree': 5
}

# Function to convert response to numeric value
def convert_response_to_numeric(response):
    return response_mapping.get(response, 0)

# Extract scale responses and name for each row
results = []
for index, row in df.iterrows():
    # Extract name
    name = row['Name:']

    # Extract scale responses
    scale_responses = row.drop(['Timestamp', 'Name:']).apply(convert_response_to_numeric)

    # Calculate sum for domain scales
    domain_scale_sum = {}
    for scale, questions in domain_scales.items():
        total = sum([scale_responses[question] for question in questions])
        domain_scale_sum[scale] = total / len(questions)  # Divide by number of questions in the scale

    # Calculate sum for facet scales
    facet_scale_sum = {}
    for scale, questions in facet_scales.items():
        total = sum([scale_responses[question] for question in questions])
        facet_scale_sum[scale] = total / len(questions)  # Divide by number of questions in the scale

    # Map domain scale averages to categories
    domain_scale_categories = {scale: map_to_category(avg) for scale, avg in domain_scale_sum.items()}

    # Map facet scale averages to categories
    facet_scale_categories = {scale: map_to_category(avg) for scale, avg in facet_scale_sum.items()}

    # Append results to list
    result = {**domain_scale_categories, **facet_scale_categories}
    result['Name:'] = name
    results.append(result)

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Print results
print(results_df)


KeyError: '1'

In [ ]:


# Define mapping for converting responses to numeric values
response_mapping = {
    'Strongly Disagree': 1,
    'Disagree': 2,
    'Neutral': 3,
    'Agree': 4,
    'Strongly Agree': 5
}

# Function to convert response to numeric value
def convert_response_to_numeric(response):
    return response_mapping.get(response, 0)

# Extract scale responses and name for each row
results = []
for index, row in df.iterrows():
    # Extract name
    name = row['Name:']

    # Extract scale responses
    scale_responses = row.drop(['Timestamp', 'Name:']).apply(convert_response_to_numeric)

    # Calculate sum for domain scales
    domain_scale_sum = {}
    for scale, questions in domain_scales.items():
        total = sum([scale_responses[question] for question in questions if question in scale_responses])
        domain_scale_sum[scale] = total / len(questions)  # Divide by number of questions in the scale

    # Calculate sum for facet scales
    facet_scale_sum = {}
    for scale, questions in facet_scales.items():
        total = sum([scale_responses[question] for question in questions if question in scale_responses])
        facet_scale_sum[scale] = total / len(questions)  # Divide by number of questions in the scale

    # Map domain scale averages to categories
    domain_scale_categories = {scale: map_to_category(avg) for scale, avg in domain_scale_sum.items()}

    # Map facet scale averages to categories
    facet_scale_categories = {scale: map_to_category(avg) for scale, avg in facet_scale_sum.items()}

    # Append results to list
    result = {**domain_scale_categories, **facet_scale_categories}
    result['Name:'] = name
    results.append(result)

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Print results
print(results_df)


  Extraversion Agreeableness Conscientiousness Negative Emotionality  \
0      Unknown       Unknown           Unknown               Unknown   
1      Unknown       Unknown           Unknown               Unknown   

  Open-Mindedness Sociability Assertiveness Energy Level Compassion  \
0         Unknown     Unknown       Unknown      Unknown    Unknown   
1         Unknown     Unknown       Unknown      Unknown    Unknown   

  Respectfulness  ... Organization Productiveness Responsibility  Anxiety  \
0        Unknown  ...      Unknown        Unknown        Unknown  Unknown   
1        Unknown  ...      Unknown        Unknown        Unknown  Unknown   

  Depression Emotional Volatility Intellectual Curiosity  \
0    Unknown              Unknown                Unknown   
1    Unknown              Unknown                Unknown   

  Aesthetic Sensitivity Creative Imagination  Name:  
0               Unknown              Unknown  Masum  
1               Unknown              Unknown   i